In [1]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, KFold
import sys
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tabulate import tabulate
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import geometric_mean_score
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, KFold
import sys
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tabulate import tabulate
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from deslib.dcs import OLA
# from deslib.dcs import KNORA_U, KNORA_E
from deslib.des import  KNOP, METADES
from deslib.static import SingleBest, StackedClassifier
from deslib.static import StaticSelection
from imblearn.metrics import geometric_mean_score

In [ ]:
x = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy")
y = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy")

In [ ]:
x_all = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy")
y_all = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy")
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all, random_state=42)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(103210, 8) (103210,)
(25803, 8) (25803,)


In [ ]:
baggings = {
    'DT': DecisionTreeClassifier(criterion='gini'),
    'NB': GaussianNB(),
    'Knn': KNeighborsClassifier(n_neighbors=7),
    'MLP': MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)
}

In [ ]:
for modelName, model in baggings.items():
    for i in tqdm(range(30)):
    # Criando o diretorio
        model_dir = os.path.join("E:/DrebinStudy/Balanced/Bagging2", modelName)
        model_dir = os.path.join(model_dir, str(i+1))
        os.makedirs(model_dir, exist_ok=True)  # Create the directory if it doesn't exist

        for j in tqdm(range(30)):

            print("entrou aqui")
            # Gerar uma amostra bootstrap
            x_resampled, y_resampled = resample(x_train, y_train, replace=True)

            # Balancear a amostra usando SMOTE
            smote = SMOTE(sampling_strategy='auto', k_neighbors=5)
            x_balanced, y_balanced = smote.fit_resample(x_resampled, y_resampled)

            print("ta treinando")

            # Treinar o modelo na amostra balanceada
            model.fit(x_balanced, y_balanced)
            
            # Salva o modelo treinado
            model_path = os.path.join(model_dir, f'model_{j+1}.joblib')
            joblib.dump(model, model_path)

In [ ]:
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []
gmean_scores = []
mcc_scores = []
cohen_scores = []


for j in tqdm(range(30)):
    # Criando o diretorio
    model_dir = os.path.join("E:/DrebinStudy/Balanced/Bagging2", modelName)
    model_dir = os.path.join(model_dir, str(j+1))

    loaded_estimators = []
    for i in range(30):
        model_path = os.path.join(model_dir, f'model_{i+1}.joblib')
        Bagging = joblib.load(model_path)
        loaded_estimators.append(Bagging)

    # 5. Fazer a previsão combinada
    y_preds = np.array([estimator.predict(x_test) for estimator in loaded_estimators])
    previsoes = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=y_preds)

    # Avaliar as métricas do modelo e armazenar na lista temporaria
    cm = confusion_matrix(y_test, previsoes)
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    geoMedia = geometric_mean_score(y_test, previsoes)
    mcc = matthews_corrcoef(y_test, previsoes)
    kappa = cohen_kappa_score(y_test, previsoes)

    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)
    gmean_scores.append(geoMedia)
    mcc_scores.append(mcc)
    cohen_scores.append(kappa)

    print(f'Repetição {i+1} {modelName} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Criar um DataFrame com as métricas
    metrics_df = pd.DataFrame({
        'Accuracy': [np.mean(acuracias)],
        'Precision': [np.mean(precisoes)],
        'Recall': [np.mean(recalls)],
        'F1_score': [np.mean(f1_scores)],
        'Roc_auc': [np.mean(roc_auc_scores)],
        'G-Mean': [np.mean(gmean_scores)],
        'MCC': [np.mean(mcc_scores)],
        'Cohen_Kappa': [np.mean(cohen_scores)],
        'Accuracy_std': [np.std(acuracias)],
        'Precision_std': [np.std(precisoes)],
        'Recall_std': [np.std(recalls)],
        'F1_score_std': [np.std(f1_scores)],
        'Roc_auc_std': [np.std(roc_auc_scores)],
        'G-Mean_std': [np.std(gmean_scores)],
        'MCC_std': [np.std(mcc_scores)],
        'Cohen_Kappa_std': [np.std(cohen_scores)],
    })

csv_path = os.path.join("E:/DrebinStudy/Balanced/Metrics", f'bagging{modelName}-2_metrics.csv')
metrics_df.to_csv(csv_path, index=False)

  3%|▎         | 1/30 [00:00<00:19,  1.47it/s]

Repetição 30 MLP - Acurácia: 0.95, Precisão: 0.48, Recall: 0.92, F1-score: 0.63, ROC AUC: 0.94


  7%|▋         | 2/30 [00:01<00:19,  1.46it/s]

Repetição 30 MLP - Acurácia: 0.95, Precisão: 0.47, Recall: 0.92, F1-score: 0.62, ROC AUC: 0.94


 10%|█         | 3/30 [00:02<00:18,  1.46it/s]

...

Repetição 30 MLP - Acurácia: 0.95, Precisão: 0.47, Recall: 0.92, F1-score: 0.62, ROC AUC: 0.93


 97%|█████████▋| 29/30 [00:19<00:00,  1.53it/s]

Repetição 30 MLP - Acurácia: 0.95, Precisão: 0.48, Recall: 0.92, F1-score: 0.63, ROC AUC: 0.94


100%|██████████| 30/30 [00:20<00:00,  1.46it/s]

Repetição 30 MLP - Acurácia: 0.95, Precisão: 0.46, Recall: 0.91, F1-score: 0.61, ROC AUC: 0.93
